## Imports necessários

In [11]:
from pymongo import MongoClient
import pymongoarrow
from pymongoarrow.api import Schema
import pandas
from pymongoarrow.monkey import patch_all
from datetime import datetime
import pyarrow as pa
patch_all()

## Conexão com o mongodb

In [12]:
client = MongoClient("mongodb://mongo1:27017,mongo2:27017,mongo3:27017/loja?replicaSet=db-replica-set&readPreference=secondaryPreferred&readPreferenceTags=dc:SP")

In [13]:
client.list_database_names()

['admin', 'config', 'local', 'loja']

### Configuração do banco de dados e collection

In [14]:
db = client["loja"]
colecao = db["produtos"]


## Convertendo para o Pandas os documentos do Mongodb

In [15]:
df= colecao.find_pandas_all({"nomeProduto": "Produto 19"})

In [17]:
print(df.head())

                        _id  idProduto nomeProduto  valorProduto  \
0  67b531141a78374c04d16272         20  Produto 19       1938.97   

           idSkus                                               skus  \
0  [66987, 69227]  [{'idSku': 66987, 'nome': 'Produto 19-0', 'val...   

                                  marcas  \
0  [{'idMarca': 10, 'nome': 'Marca 50'}]   

                                          categorias  \
0  [{'idCategoria': 10, 'nome': 'Categoria 2'}, {...   

                                             imagens  
0  [{'idImagem': 4077, 'url': 'https://example.co...  


## Convertendo para o Apache Arrow os documentos do Mongodb

In [18]:
arrow_table = colecao.find_arrow_all({"nomeProduto": "Produto 19"})

In [19]:
print(arrow_table)

pyarrow.Table
_id: extension<pymongoarrow.objectid<ObjectIdType>>
idProduto: int32
nomeProduto: string
valorProduto: double
idSkus: list<item: int32>
  child 0, item: int32
skus: list<item: struct<idSku: int32, nome: string, valor: double, estoque: int32, cor: string>>
  child 0, item: struct<idSku: int32, nome: string, valor: double, estoque: int32, cor: string>
      child 0, idSku: int32
      child 1, nome: string
      child 2, valor: double
      child 3, estoque: int32
      child 4, cor: string
marcas: list<item: struct<idMarca: int32, nome: string>>
  child 0, item: struct<idMarca: int32, nome: string>
      child 0, idMarca: int32
      child 1, nome: string
categorias: list<item: struct<idCategoria: int32, nome: string>>
  child 0, item: struct<idCategoria: int32, nome: string>
      child 0, idCategoria: int32
      child 1, nome: string
imagens: list<item: struct<idImagem: int32, url: string>>
  child 0, item: struct<idImagem: int32, url: string>
      child 0, idImagem: i

## Configurando o schema para o resultado

In [20]:
# Definir um Schema para os produtos
produto_schema = Schema({
    "idProduto": pa.int32(),
    "nomeProduto": pa.string(),
    "valorProduto": pa.float64(),
    "idSkus": pa.list_(pa.int32()),  # Array de inteiros
    "skus": pa.list_(pa.struct({     # Array de JSONs embutidos
        "idSku": pa.int32(),
        "nome": pa.string(),
        "valor": pa.float64()
    })),
     "categorias": pa.list_(pa.struct({     # Array de JSONs embutidos
        "idCategoria": pa.int32(),
        "nome": pa.string()     
    })),  # Array de strings
     "marcas": pa.list_(pa.struct({     # Array de JSONs embutidos
        "idMarca": pa.int32(),
        "nome": pa.string()     
    })),  # Array de strings
    "imagens": pa.list_(pa.struct({   # Array de JSONs para imagens
        "url": pa.string()      
    }))
})


In [21]:
df= colecao.find_pandas_all({"nomeProduto": "Produto 19"},schema=produto_schema)

In [22]:
# Exibir a estrutura do schema retornado
print(df)

   idProduto nomeProduto  valorProduto          idSkus  \
0         20  Produto 19       1938.97  [66987, 69227]   

                                                skus  \
0  [{'idSku': 66987, 'nome': 'Produto 19-0', 'val...   

                                          categorias  \
0  [{'idCategoria': 10, 'nome': 'Categoria 2'}, {...   

                                  marcas  \
0  [{'idMarca': 10, 'nome': 'Marca 50'}]   

                                             imagens  
0  [{'url': 'https://example.com/img40.jpg'}, {'u...  


## Consultando as informações com agregações

In [23]:
df = colecao.aggregate_pandas_all([{ "$unwind": "$skus" },{ "$project": {"_id":0, "nomeProduto": 1, "idSku": "$skus.idSku", "nomeSku": "$skus.nome", "valor": "$skus.valor" } }])

In [24]:
print(df)

      nomeProduto  idSku        nomeSku    valor
0       Produto 7  93301    Produto 7-0  4293.02
1       Produto 7  67922    Produto 7-1  4859.48
2       Produto 7  77185    Produto 7-2  4563.95
3      Produto 39  89282   Produto 39-0  4681.97
4      Produto 39  30299   Produto 39-1   660.60
...           ...    ...            ...      ...
1580  Produto 493  38398  Produto 493-0   270.40
1581  Produto 493  91055  Produto 493-1  3749.68
1582  Produto 493  75592  Produto 493-2   669.25
1583  Produto 493  84598  Produto 493-3  4673.84
1584  Produto 493  64779  Produto 493-4  1845.09

[1585 rows x 4 columns]


## Escrevendo o resultado em uma coleção do Mongodb

In [25]:
from pymongoarrow.api import write
colecao_sku = db["skus"]
write(colecao_sku, df)

{'insertedCount': 1585}